In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np 

review_data = pd.read_csv("/content/drive/My Drive/cmpe 255/yelp_review.csv")

In [3]:
busi_data = pd.read_csv("/content/drive/My Drive/cmpe 255/yelp_business.csv")

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (49) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
review_data = review_data[['business_id', 'text', 'stars']]

print(review_data.shape)
review_data.head()

(418837, 3)


,business_id,text,stars
0,buF9druCkbuXLX526sGELQ,Apparently Prides Osteria had a rough summer a...,4.0
1,RA4V8pr014UyUbDvI-LW2A,This store is pretty good. Not as great as Wal...,4.0
2,_sS2LBIGNT5NQb6PD1Vtjw,I called WVM on the recommendation of a couple...,5.0
3,0AzLzHfOJgL7ROwhdww2ew,I've stayed at many Marriott and Renaissance M...,2.0
4,8zehGz9jnxPqXtOc7KaJxA,The food is always great here. The service fro...,4.0


In [5]:
print(busi_data.shape)
busi_data = busi_data[busi_data.categories.str.contains('restaurant|food', na=False)]

print(busi_data.shape)
busi_data = busi_data[['business_id', 'name']]
busi_data.head()

(160585, 60)
(3108, 60)


,business_id,name
29,jGennaZUr2MsJyRhijNBfA,Legal Sea Foods
36,jx91IMdGOmLOo8h_F9z39g,Cleary's Restaurant & Spirits
41,NRPemqVb4qpWFF0Avq_6OQ,Eurasia Sushi Bar & Seafood
74,Mf9io8WkMyGautXuSSfGTg,Los Potrillos Mexican Restaurant & Bar
112,NoPXKdIwRzflOW-5exFAqA,1st Avenue Seafood by Pocha


In [6]:
# data = data[data.business_id.str.contains()]

In [7]:
review_data['business_id'] = review_data['business_id'].astype(str)
busi_data['business_id'] = busi_data['business_id'].astype(str)

data = review_data.merge(busi_data, on='business_id')
print(data.shape)
data.head()

(31447, 4)


,business_id,text,stars,name
0,EEHhKSxUvJkoPSzeGKkpVg,I work in the Pru and this is the most afforda...,5.0,Boston Chowda Co
1,EEHhKSxUvJkoPSzeGKkpVg,How do I love thee Chowda? Let me count the wa...,4.0,Boston Chowda Co
2,EEHhKSxUvJkoPSzeGKkpVg,"Oy. Food poisoning two times in a row, months ...",1.0,Boston Chowda Co
3,EEHhKSxUvJkoPSzeGKkpVg,Two Stars. Here is why - Quality Control. \n\...,2.0,Boston Chowda Co
4,EEHhKSxUvJkoPSzeGKkpVg,This place is a must! I recommend the lobster ...,4.0,Boston Chowda Co


In [8]:
# counting null values in the dataset

data.isnull().sum()

business_id    0
text           0
stars          0
name           0
dtype: int64

In [9]:
import nltk 
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

data['text'] = data['text'].astype(str)
#print(data.head())
data['text'] = data['text'].str.lower()
#print(data.head())

# stop word removal
from nltk.corpus import stopwords
stop = stopwords.words('english')

data['text'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
data.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,business_id,text,stars,name
0,EEHhKSxUvJkoPSzeGKkpVg,work pru affordable tasty place food court. de...,5.0,Boston Chowda Co
1,EEHhKSxUvJkoPSzeGKkpVg,love thee chowda? let count ways love thy deli...,4.0,Boston Chowda Co
2,EEHhKSxUvJkoPSzeGKkpVg,"oy. food poisoning two times row, months apart...",1.0,Boston Chowda Co
3,EEHhKSxUvJkoPSzeGKkpVg,two stars. - quality control. lobster bisque u...,2.0,Boston Chowda Co
4,EEHhKSxUvJkoPSzeGKkpVg,place must! recommend lobster roll (stuffed lo...,4.0,Boston Chowda Co


In [10]:
import string
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet, stopwords
from nltk import pos_tag, word_tokenize
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import WhitespaceTokenizer
import re

def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:         
        return None
    
    
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    # lemmatize text
    pos_tagged = nltk.pos_tag(nltk.word_tokenize(text))

    wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
    # print(wordnet_tagged)

    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            # if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:       
            # else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    text = " ".join(lemmatized_sentence)
    return(text)


def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)


def clean_text(text):
    # remove html tags
    text = remove_html_tags(text)
    # lower text
    text = text.lower()
    # unicode characters
    text = text.encode('ascii', 'ignore').decode()
    # remove digits
    text = ''.join([i for i in text if not i.isdigit()])
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # remove stop words
    stop_words = stopwords.words('english')
    text = ' '.join([word for word in text.split(' ') if word not in stop_words])
    return text

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [11]:
data['text'] = data.text.apply(clean_text)
data.head()

,business_id,text,stars,name
0,EEHhKSxUvJkoPSzeGKkpVg,work pru affordable tasty place food court dea...,5.0,Boston Chowda Co
1,EEHhKSxUvJkoPSzeGKkpVg,love thee chowda let count ways love thy delic...,4.0,Boston Chowda Co
2,EEHhKSxUvJkoPSzeGKkpVg,oy food poisoning two times row months apart c...,1.0,Boston Chowda Co
3,EEHhKSxUvJkoPSzeGKkpVg,two stars quality control lobster bisque used...,2.0,Boston Chowda Co
4,EEHhKSxUvJkoPSzeGKkpVg,place must recommend lobster roll stuffed lobs...,4.0,Boston Chowda Co


In [14]:
words = ["discomfort", "ache", "death", "allergy", "diarrhoea", "infection", "stomach", "loosemotion", "sick", "vomit", "nausea", "throw up", "threw up", "unhygienic", "pain", "flu", "stale", "poision", "motion", "upset", "indigestion", "acidity", "barf" ]

for w in words:
  w = lemmatize_text(w);

In [15]:
print(words)

['discomfort', 'ache', 'death', 'allergy', 'diarrhoea', 'infection', 'stomach', 'loosemotion', 'sick', 'vomit', 'nausea', 'throw up', 'threw up', 'unhygienic', 'pain', 'flu', 'stale', 'poision', 'motion', 'upset', 'indigestion', 'acidity', 'barf']


In [16]:
# creating a new feature for food poisoning based on words seen in review text
for index,row in data.iterrows() :
    if any(word in row['text'] for word in words) and row['stars'] < 3:
        data.loc[index,'FP'] = 1
    else:
        data.loc[index,'FP'] = 0


data.head()

,business_id,text,stars,name,FP
0,EEHhKSxUvJkoPSzeGKkpVg,work pru affordable tasty place food court dea...,5.0,Boston Chowda Co,0.0
1,EEHhKSxUvJkoPSzeGKkpVg,love thee chowda let count ways love thy delic...,4.0,Boston Chowda Co,0.0
2,EEHhKSxUvJkoPSzeGKkpVg,oy food poisoning two times row months apart c...,1.0,Boston Chowda Co,0.0
3,EEHhKSxUvJkoPSzeGKkpVg,two stars quality control lobster bisque used...,2.0,Boston Chowda Co,0.0
4,EEHhKSxUvJkoPSzeGKkpVg,place must recommend lobster roll stuffed lobs...,4.0,Boston Chowda Co,0.0


In [17]:
fp_data = data[data['FP'] == 1.0] 
print(fp_data.shape)
fp_data.head()

(608, 5)


,business_id,text,stars,name,FP
117,zmZ3HkVCeZPBefJJxzdJ7A,ehh food good mean tasted good took half hour...,1.0,Le's Vietnamese Resturant,1.0
125,zmZ3HkVCeZPBefJJxzdJ7A,dining companion told allergic peanuts served ...,1.0,Le's Vietnamese Resturant,1.0
138,zmZ3HkVCeZPBefJJxzdJ7A,wow food decent service utterly terrible notic...,1.0,Le's Vietnamese Resturant,1.0
215,zmZ3HkVCeZPBefJJxzdJ7A,although used love les really pouring msg lett...,1.0,Le's Vietnamese Resturant,1.0
294,zmZ3HkVCeZPBefJJxzdJ7A,friend got sick full week eating beef pho never,1.0,Le's Vietnamese Resturant,1.0


In [18]:
from sklearn.model_selection import train_test_split

# split the data into train and test set
train, test = train_test_split(data, test_size=0.2, random_state=42, shuffle=True)

In [19]:
train_X = train['text']  
train_y = train['FP']   
test_X = test['text'] 
test_y = test['FP']

In [20]:
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
t = time()  

# loading CountVectorizer
tf_vectorizer = CountVectorizer()

X_train_tf = tf_vectorizer.fit_transform(train_X)
duration = time() - t
print("Time taken to extract features from training data : %f seconds" % (duration))
print("n_samples: %d, n_features: %d" % X_train_tf.shape)

Time taken to extract features from training data : 1.346545 seconds
n_samples: 25157, n_features: 42138


In [21]:
t = time()
X_test_tf = tf_vectorizer.transform(test_X)

duration = time() - t
print("Time taken to extract features from test data : %f seconds" % (duration))
print("n_samples: %d, n_features: %d" % X_test_tf.shape)

Time taken to extract features from test data : 0.334381 seconds
n_samples: 6290, n_features: 42138


In [22]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
# build naive bayes classification model
t = time()

naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tf, train_y)

training_time = time() - t
print("train time: %0.3fs" % training_time)

train time: 0.037s


In [23]:
# predict the testing dataset
t = time()
y_pred = naive_bayes_classifier.predict(X_test_tf)

In [24]:
y_pred = pd.DataFrame(y_pred)
test = pd.concat([test,y_pred])

count = test.groupby(['business_id', 'FP'], as_index=False).size()
count = pd.DataFrame(count)
print(count.head())
print(count.isnull().sum())
print()

              business_id   FP  size
0  -bjdCwpumWJIE9BsqAdGzw  0.0     5
1  -uHbc7ssOFsOnyVFjAY2Vw  0.0     4
2  0Y5XXVsq6vKu5HL5yf7p5w  0.0    22
3  1iU41Mf9FGZACZZmdaemaw  0.0    29
4  1iU41Mf9FGZACZZmdaemaw  1.0     2
business_id    0
FP             0
size           0
dtype: int64



In [25]:
percentages = count.groupby('business_id').agg({'size': 'sum'})
print(percentages)

                        size
business_id                 
-bjdCwpumWJIE9BsqAdGzw     5
-uHbc7ssOFsOnyVFjAY2Vw     4
0Y5XXVsq6vKu5HL5yf7p5w    22
1iU41Mf9FGZACZZmdaemaw    31
1x_ugDgc57dAkftUbUXIvA     2
...                      ...
yXVgVKV-Ek_tE9EZXMVTEQ    37
yZ_kkTJKYGX7FEpyrROnew     3
yaWQTd2T9vXNJaw3jiSrIw    27
ynF99WDWAz3KS2vPOZoq9Q     4
zmZ3HkVCeZPBefJJxzdJ7A    88

[257 rows x 1 columns]


In [26]:
pp = percentages.merge(count, on='business_id')
#print(pp)

is_FP1 =  pp['FP']==1.0
final_perc = pp[is_FP1]
final_perc

,business_id,size_x,FP,size_y
4,1iU41Mf9FGZACZZmdaemaw,31,1.0,2
12,2trWtHH6UCCqP8FIfnoYig,16,1.0,1
18,3_FEKPQLogDrzzSZLb7WyA,10,1.0,2
22,4UjU7F_EX3lgUtkzN8Bbrw,46,1.0,3
27,5HAuhk6_h5t7j6fk983QZg,61,1.0,1
...,...,...,...,...
305,vPXrpTMN3Z4IjcX4ZbrlJQ,37,1.0,2
311,xLgBZ_rTJ8QnpsqQwphtEQ,34,1.0,1
321,yXVgVKV-Ek_tE9EZXMVTEQ,37,1.0,1
325,ynF99WDWAz3KS2vPOZoq9Q,4,1.0,2


In [27]:
Bus_prob = []

for index,row in final_perc.iterrows():
  percent = row['size_y'] / row['size_x']
  #print(df)
  Bus_prob.append({'business_id': row['business_id'], 'Percentage': round(percent*100,2)})

Bus_prob = pd.DataFrame(Bus_prob)
pd.set_option("display.max_rows", None, "display.max_columns", None)

sorted_prob = Bus_prob.sort_values(by = 'Percentage', ascending=False)

In [28]:
sorted_prob

,business_id,Percentage
69,ynF99WDWAz3KS2vPOZoq9Q,50.00
14,7jJZ91A-tkOghVOrj3aE6A,50.00
32,OS0kKNR7NQY1AZEw4XYCdg,25.00
2,3_FEKPQLogDrzzSZLb7WyA,20.00
54,jvnuFNDtIxVDBrm2wIOiRA,17.65
18,9yZpfG9uP0DjDHSUQokfvw,16.67
34,Pkp824mx2LL93TqKOdfMyQ,14.29
44,Z3lgc-YvQf80JOOV0NNI4A,13.33
23,DeKd_nL_AeEJdi5RmRMmeg,12.50
41,TySByjFQEjj-rU39L5e_aQ,12.50


In [29]:
Final_FP = sorted_prob[sorted_prob["Percentage"] > 30.00]
print(Final_FP)
print(Final_FP.shape[0]," restaurants have more than 30% chance of giving Food Poisoning to their customers")

               business_id  Percentage
69  ynF99WDWAz3KS2vPOZoq9Q        50.0
14  7jJZ91A-tkOghVOrj3aE6A        50.0
2  restaurants have more than 30% chance of giving Food Poisoning to their customers
